In [2]:
import sqlalchemy as sq
from sqlite3 import dbapi2 as sqlite
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Product(Base):
    __tablename__ = "product"
    name = sq.Column(sq.String(200), nullable=True)
    id = sq.Column(sq.Integer, primary_key=True, autoincrement=True)

    discriminator = sq.Column(sq.String)
    __table_args__ = (sq.UniqueConstraint('discriminator', 'name', name='uix_1'),)
    
engine = sq.create_engine('sqlite+pysqlite://', module=sqlite, echo=False)

# create some tables
Base.metadata.create_all(engine)
print(sq.__version__)

1.2.17


In [3]:
from pyutil.sql.base import Base
#from pyutil.sql.interfaces.products import ProductInterface
from pyutil.sql.interfaces.symbols.symbol import Symbol#, Base

engine = sq.create_engine('sqlite+pysqlite://', module=sqlite, echo=True)
# create some tables
Base.metadata.create_all(engine)

from sqlalchemy.orm import Session

connection = engine.connect()
session = Session(bind=connection)


2019-02-26 17:16:00,972 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-26 17:16:00,972 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:16:00,973 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-26 17:16:00,973 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:16:00,974 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reference_field")
2019-02-26 17:16:00,974 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:16:00,975 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reference_data")
2019-02-26 17:16:00,975 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:16:00,975 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("productinterface")
2019-02-26 17:16:00,976 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:16:00,976 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("series")
2019-02-26 17:16:01,017 INFO sqlalchemy.engine.base.Engine ()
2019-02-26 17:1

In [4]:
from pyutil.sql.interfaces.symbols.symbol import Symbol, SymbolType
symbol = Symbol(name="A", group=SymbolType.equities, internal="Peter Maffay")

session.add(symbol)
session.add(symbol)
session.commit()

for symbol in session.query(Symbol):
    print(symbol)
    
session.add(symbol)
session.commit()

2019-02-26 17:16:01,050 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-26 17:16:01,052 INFO sqlalchemy.engine.base.Engine INSERT INTO productinterface (name, discriminator) VALUES (?, ?)
2019-02-26 17:16:01,053 INFO sqlalchemy.engine.base.Engine ('A', 'symbol')
2019-02-26 17:16:01,055 INFO sqlalchemy.engine.base.Engine INSERT INTO symbol (id, "group", internal) VALUES (?, ?, ?)
2019-02-26 17:16:01,055 INFO sqlalchemy.engine.base.Engine (1, 'equities', 'Peter Maffay')
2019-02-26 17:16:01,056 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-26 17:16:01,059 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-26 17:16:01,060 INFO sqlalchemy.engine.base.Engine SELECT productinterface.name AS productinterface_name, symbol.id AS symbol_id, productinterface.id AS productinterface_id, productinterface.discriminator AS productinterface_discriminator, symbol."group" AS symbol_group, symbol.internal AS symbol_internal, reference_data_1.field_id AS reference_data_1_field_id, r